# EasyVVUQ in R

**Louise Kimpton**, Vytautas Jancauskas

In this tutorial we will use the ```reticulate``` package in R to interface with EasyVVUQ. This tutorial is similar to the basic tutorial in EasyVVUQ. The idea here is that you would use R to generate a sampling plan for EasyVVUQ to manage and execute and then retrieve the simulation results back into R. To this end we don't use any of the EasyVVUQ sampling and analysis tools.

We will use the [reticulate](https://rstudio.github.io/reticulate/) package which allows one to interact with a running Python instance and exchange data frames between R and pandas.

In [1]:
install.packages("reticulate")

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [2]:
library("reticulate")

We then import the easyvvuq packages we will need in this tutorial.

In [3]:
uq <- import('easyvvuq')
actions <- import('easyvvuq.actions')
sampling <- import('easyvvuq.sampling')

We define input parameters as we would normally.

In [4]:
params <- dict(
  'F' = dict('type' = 'float', 'default' = 1.0),
  'L' = dict('type' = 'float', 'default' = 1.5),
  'a' = dict('type' = 'float', 'min' = 0.7, 'max' = 1.2, 'default' = 1.0),
  'D' = dict('type' = 'float', 'min' = 0.75, 'max' = 0.85, 'default' = 0.8),
  'd' = dict('type' = 'float', 'default' = 0.1),
  'E' = dict('type' = 'float', 'default' = 200000),
  'outfile' = dict('type' = 'string', 'default' = 'output.json')
)

Creating the campaign is again the same.

In [5]:
encoder <- uq$encoders$GenericEncoder(template_fname='beam.template', delimiter='$', target_filename='input.json')
decoder <- uq$decoders$JSONDecoder(target_filename='output.json', output_columns=list('g1')) 
execute <- actions$ExecuteLocal(paste(getwd(), '/beam input.json', sep=""))
actions <- actions$Actions(actions$CreateRunDirectory('/tmp'), 
                           actions$Encode(encoder), execute, actions$Decode(decoder))

In [6]:
campaign <- uq$Campaign(name='beam', params=params, actions=actions)

We specify a data frame with the points we will use to evaluate the model on. In this case three points will be evaluated. This data frame represents a sampling plan as required to use whatever analysis method you want.

In [7]:
df <- data.frame(
  'F' = c(1.145, 1.029, 0.961),
  'L' = c(1.495, 1.508, 1.501),
  'a' = c(0.782, 0.899, 1.141),
  'D' = c(0.795, 0.802, 0.816)
)

This is then used as an argument to ```DataFrameSampler```.

In [8]:
campaign$set_sampler(sampling$DataFrameSampler(df=df))

In [9]:
campaign$execute()$collate()

Finally we can retrieve the results as usual.

In [10]:
campaign$get_collation_result()

run_id 0,iteration 0,F 0,L 0,a 0,D 0,d 0,E 0,outfile 0,g1 0
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>
1,0,1.145,1.495,0.782,0.795,0.1,2e+05,output.json,-1.012152e-05
2,0,1.029,1.508,0.899,0.802,0.1,2e+05,output.json,-8.395071e-06
3,0,0.961,1.501,1.141,0.816,0.1,2e+05,output.json,-4.137187e-06
